## Merging hourly taxi, weather, and incoming passenger data for 2017

In [1]:
import numpy as np
from tqdm import tqdm
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

### Reading Aggregated Taxi Data

In [2]:
taxi_hourly_df = pd.read_csv('../clean_data/Aggregated_TaxiData_14-17.csv', index_col=0)

In [3]:
taxi_hourly_df.head()

,num_pickups,num_passengers
tpep_pickup_datetime,,
2014-01-01 00:00:00,107,210.0
2014-01-01 01:00:00,34,45.0
2014-01-01 02:00:00,34,70.0
2014-01-01 03:00:00,12,18.0
2014-01-01 04:00:00,25,37.0


In [4]:
taxi_hourly_df.shape

(26009, 2)

In [5]:
taxi_hourly_df.index = pd.DatetimeIndex(taxi_hourly_df.index)

In [6]:
taxi_hourly_df = taxi_hourly_df.drop([pd.Timestamp('2014-11-02 01:00:00'), pd.Timestamp('2015-11-01 01:00:00'), pd.Timestamp('2017-03-12 02:00:00')])#, pd.Timestamp('2017-03-12 02:00:00')])
taxi_hourly_df.index = taxi_hourly_df.index.tz_localize('America/New_York')

In [7]:
taxi_hourly_df.index.min()

Timestamp('2014-01-01 00:00:00-0500', tz='America/New_York')

In [8]:
taxi_hourly_df.index.max()

Timestamp('2017-06-30 23:00:00-0400', tz='America/New_York')

In [9]:
taxi_hourly_df.shape

(26006, 2)

### Adding Federal Calendar Holidays

In [10]:
dr = pd.to_datetime(pd.to_datetime(taxi_hourly_df.index.date))

cal = calendar()
holidays = cal.holidays(start=dr.min(), end=dr.max())

taxi_hourly_df['holiday'] = dr.isin(holidays)

In [11]:
taxi_hourly_df.shape

(26006, 3)

In [12]:
taxi_hourly_df.head()

,num_pickups,num_passengers,holiday
tpep_pickup_datetime,,,
2014-01-01 00:00:00-05:00,107,210.0,True
2014-01-01 01:00:00-05:00,34,45.0,True
2014-01-01 02:00:00-05:00,34,70.0,True
2014-01-01 03:00:00-05:00,12,18.0,True
2014-01-01 04:00:00-05:00,25,37.0,True


### Checking Missing Hours from Data and adding them to Datetimeindex. Also created a binary column as Indicator

In [13]:
dateindex = pd.date_range(taxi_hourly_df.index.min(), taxi_hourly_df.index.max(), freq="1H")

In [14]:
taxi_hourly_df = taxi_hourly_df.reindex(dateindex)

In [15]:
taxi_hourly_df.head()

,num_pickups,num_passengers,holiday
2014-01-01 00:00:00-05:00,107.0,210.0,True
2014-01-01 01:00:00-05:00,34.0,45.0,True
2014-01-01 02:00:00-05:00,34.0,70.0,True
2014-01-01 03:00:00-05:00,12.0,18.0,True
2014-01-01 04:00:00-05:00,25.0,37.0,True


In [16]:
taxi_hourly_df["missing_dt"] = np.NaN

In [17]:
taxi_hourly_df.loc[taxi_hourly_df.isnull().all(1) == True, "missing_dt"] = True
taxi_hourly_df.loc[~(taxi_hourly_df.missing_dt == True), "missing_dt"] = False

In [19]:
taxi_hourly_df[taxi_hourly_df.missing_dt == True].head()

,num_pickups,num_passengers,holiday,missing_dt
2014-01-20 02:00:00-05:00,NaN,NaN,NaN,True
2014-01-21 03:00:00-05:00,NaN,NaN,NaN,True
2014-01-22 03:00:00-05:00,NaN,NaN,NaN,True
2014-02-12 03:00:00-05:00,NaN,NaN,NaN,True
2014-02-17 03:00:00-05:00,NaN,NaN,NaN,True


### Weather Data

In [117]:
def clean_weather(path):
    weather_df = pd.read_csv(path)
    weather_df["date"] = pd.DatetimeIndex(pd.to_datetime(weather_df.dt, unit="s")) \
                    .tz_localize('UTC').tz_convert('America/New_York')
    weather_df.drop_duplicates(['date'], keep='first', inplace = True)
    weather_df.set_index("date", inplace=True)
    
    weather_df.drop(["dt", "dt_iso", "city_id", "city_name", "lat", "lon", "weather_icon", "grnd_level", \
                 "sea_level", "clouds_all", "weather_id", "rain_1h", "rain_3h", "rain_24h", \
                 "rain_today", "snow_1h", "snow_3h", "snow_24h", "snow_today"], axis=1, inplace=True)
    return weather_df

In [118]:
weather_df = clean_weather("../clean_data/weather_NY.csv")

### Merge Taxi and Weather Data

In [119]:
taxi_weather = taxi_hourly_df.merge(right=weather_df, how='left', left_index=True, right_index=True)
taxi_weather.index = taxi_weather.index.tz_convert('America/New_York')

In [120]:
taxi_weather_filled = taxi_weather.fillna(method='ffill')

In [121]:
taxi_weather_filled.head()

,num_pickups,num_passengers,holiday,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,weather_main,weather_description
tpep_pickup_datetime,,,,,,,,,,,,
2014-01-01 00:00:00-05:00,107,210.0,True,269.27,267.59,270.93,1026.0,80.0,3.0,313.0,Clear,Sky is Clear
2014-01-01 01:00:00-05:00,34,45.0,False,268.90,267.04,270.93,1026.0,80.0,2.0,293.0,Clear,Sky is Clear
2014-01-01 02:00:00-05:00,34,70.0,False,268.48,266.48,270.37,1026.0,57.0,3.0,244.0,Clear,Sky is Clear
2014-01-01 03:00:00-05:00,12,18.0,False,268.33,265.93,270.37,1024.0,80.0,3.0,244.0,Clear,Sky is Clear
2014-01-01 04:00:00-05:00,25,37.0,False,268.10,265.37,270.37,1024.0,80.0,3.0,244.0,Clear,Sky is Clear


### Passenger seat data

In [122]:
def clean_seats(path):
    seats_per_hour = pd.read_csv(path, header=None)
#     print(seats_per_hour.head())
    seats_per_hour.columns = ['Time', 'Passengers']
    seats_per_hour["Time"] = pd.DatetimeIndex(seats_per_hour.Time)
    seats_per_hour.set_index("Time", inplace = True)

    #To get rid of errors surrounding daylight savings time. All of taxi pickups and passenger counts are NaNs for these times
    seats_per_hour = seats_per_hour.drop([pd.Timestamp('2014-03-09 02:00:00'), pd.Timestamp('2014-11-02 01:00:00'), pd.Timestamp('2015-03-08 02:00:00'), pd.Timestamp('2015-11-01 01:00:00'), pd.Timestamp('2016-03-13 02:00:00'), pd.Timestamp('2016-11-06 01:00:00'), pd.Timestamp('2017-03-12 02:00:00')])
    seats_per_hour.index = seats_per_hour.index.tz_localize('America/New_York')
    return seats_per_hour

In [123]:
seats_per_hour = clean_seats('../clean_data/passengers_per_hour_since_2014.csv')

### Merging all dataframes

In [124]:
merged_df = pd.merge(left=taxi_weather_filled, right=seats_per_hour, how='inner', left_index=True, right_index=True)

In [125]:
merged_df.shape

(26005, 13)

In [147]:
merged_df['Month'] = merged_df.index.month

In [150]:
merged_df['Hour'] = merged_df.index.hour

In [153]:
merged_df['Day'] = merged_df.index.dayofweek

In [155]:
merged_df.head()

,num_pickups,num_passengers,holiday,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,weather_main,weather_description,Passengers,Month,Hour,Day
2014-01-01 00:00:00-05:00,107,210.0,True,269.27,267.59,270.93,1026.0,80.0,3.0,313.0,Clear,Sky is Clear,108,1,0,2
2014-01-01 01:00:00-05:00,34,45.0,False,268.90,267.04,270.93,1026.0,80.0,2.0,293.0,Clear,Sky is Clear,0,1,1,2
2014-01-01 02:00:00-05:00,34,70.0,False,268.48,266.48,270.37,1026.0,57.0,3.0,244.0,Clear,Sky is Clear,0,1,2,2
2014-01-01 03:00:00-05:00,12,18.0,False,268.33,265.93,270.37,1024.0,80.0,3.0,244.0,Clear,Sky is Clear,0,1,3,2
2014-01-01 04:00:00-05:00,25,37.0,False,268.10,265.37,270.37,1024.0,80.0,3.0,244.0,Clear,Sky is Clear,0,1,4,2


In [156]:
merged_df.to_csv("../clean_data/combined_final_data.csv")

### Checking nan hours

In [130]:
df = pd.DataFrame({'P' : np.random.rand(30625)}, index=pd.date_range('2014-01-01', '2017-06-30', freq='h'))

In [133]:
df.index = df.index.tz_localize('EST')

In [136]:
check_nans = merged_df.reindex(df.index)

In [137]:
check_nans.isnull

<bound method NDFrame.isnull of                            num_pickups  num_passengers holiday     temp  \
2014-01-01 00:00:00-05:00        107.0           210.0    True  269.270   
2014-01-01 01:00:00-05:00         34.0            45.0   False  268.900   
2014-01-01 02:00:00-05:00         34.0            70.0   False  268.480   
2014-01-01 03:00:00-05:00         12.0            18.0   False  268.330   
2014-01-01 04:00:00-05:00         25.0            37.0   False  268.100   
2014-01-01 05:00:00-05:00         16.0            24.0   False  268.020   
2014-01-01 06:00:00-05:00         14.0            17.0   False  267.900   
2014-01-01 07:00:00-05:00         21.0            35.0   False  267.590   
2014-01-01 08:00:00-05:00         94.0           190.0   False  267.670   
2014-01-01 09:00:00-05:00        164.0           285.0   False  269.000   
2014-01-01 10:00:00-05:00        379.0           673.0   False  270.650   
2014-01-01 11:00:00-05:00        367.0           703.0   False  271.

In [132]:
merged_df.head()

,num_pickups,num_passengers,holiday,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,weather_main,weather_description,Passengers
2014-01-01 00:00:00-05:00,107,210.0,True,269.27,267.59,270.93,1026.0,80.0,3.0,313.0,Clear,Sky is Clear,108
2014-01-01 01:00:00-05:00,34,45.0,False,268.90,267.04,270.93,1026.0,80.0,2.0,293.0,Clear,Sky is Clear,0
2014-01-01 02:00:00-05:00,34,70.0,False,268.48,266.48,270.37,1026.0,57.0,3.0,244.0,Clear,Sky is Clear,0
2014-01-01 03:00:00-05:00,12,18.0,False,268.33,265.93,270.37,1024.0,80.0,3.0,244.0,Clear,Sky is Clear,0
2014-01-01 04:00:00-05:00,25,37.0,False,268.10,265.37,270.37,1024.0,80.0,3.0,244.0,Clear,Sky is Clear,0
